**An update regarding our project:**


Project requirements:

- Pulse Width Modulation
- Regulate repetition rate for laser driver. 

P.S: When the laser is running in **pulsed mode** the number of pulses per second or frequency of pulses is known as the **Pulse Repetition rate (PRR)**.  The term PRR and pulse frequency are commonly used. 



Status of development:

- We are using MicroPython, which is an implementation of Pyton Language written in C, that is optimized to run on microcontrollers.
- We've been able to generate a square-wave using the STM32F4 DAC
- The DAC is operating in **triggered mode**: this is allowing the updates to be initiated by a rising edge on an internal or external synchronization signal.


Still left to be done:

- Using RC components as a low pass filter to remove noise from generated signal



**Current Implementation of Square-wave generation using DAC**

```python
import pyb
from pyb import DAC
import math
from array import array


# Define sign function, needed to generate square-wave
def sign(x): return math.copysign(1, x)

# Configure and Initialize Pin 'PA4' as analog
pin = pyb.Pin('A4', pyb.Pin.ANALOG)


# create a buffer containing a square-wave from a sine-wave, using half-word samples
# 'H'-> C (unsigned short) -> Python(int)
buf = array('H', 2048 + int(2047 * sign(math.sin(2 * math.pi * i / 128))) for i in range(128))


dac = DAC(pin, bits=12)
# output the square-wave at 800Hz
#dac.write_timed(buf, 800 * len(buf), mode=DAC.CIRCULAR)

# output the square-wave using a timer object using timer 6 triggered at 100Hz
dac.write_timed(buf, pyb.Timer(6, freq=100), mode=DAC.CIRCULAR)
```

What's happening?:

- Initiates a burst of RAM to DAC using a DMA transfer. The input data is treated as an array of bytes in 8-bit mode, and an array of unsigned half-words (array typecode ‘H’) in 12-bit mode.

- `freq` can be an integer specifying the frequency to write the DAC samples at, using `Timer(6)`. Or it can be an already-initialised Timer object which is used to trigger the DAC sample. Valid timers are 2, 4, 5, 6, 7 and 8.

- `Timer(6)`:
 - 0 channels - no direct external pin control.
 - must be Master
 - up count only with DMA control
 - 16 bit prescaler
 - used for timebase timers or triggering the DAC
 - max speed 84 MHz, max interface speed 42 MHz


- `mode` can be `DAC.NORMAL` or `DAC.CIRCULAR`

![](sine-wave.jpg)

![](square-wave.jpg)

**Resources:**
- https://github.com/micropython/micropython/wiki/PWM-Timers
- https://docs.micropython.org/en/latest/pyboard/pyboard/tutorial/timer.html?highlight=timer